In [8]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from PIL import Image
import pandas as pd
import numpy as np
from torchvision import transforms
from torch import nn
import torch.optim as optim
from tqdm import tqdm
import gdown
import timm  # Import timm for Vision Transformer models
from torchvision.transforms import InterpolationMode
from ultralytics import YOLO
from torchvision.transforms import InterpolationMode
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

 60%|█████▉    | 186M/310M [07:03<04:44, 438kB/s] 


In [2]:
# Device configuration
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

# Create data directory
os.makedirs("data", exist_ok=True)

# Download dataset using gdown
url = 'https://drive.google.com/uc?id=1IEnpbGjNqXYF4vPY1NW-ODcrYZomyb4S'
output = 'data/dataset.zip'
gdown.download(url, output, quiet=False)

# Unzip dataset
!unzip -q data/dataset.zip -d data

cuda


Downloading...
From (original): https://drive.google.com/uc?id=1IEnpbGjNqXYF4vPY1NW-ODcrYZomyb4S
From (redirected): https://drive.google.com/uc?id=1IEnpbGjNqXYF4vPY1NW-ODcrYZomyb4S&confirm=t&uuid=e99461e4-d86a-4a20-a0e7-5dc7ccbfbe27
To: /home/atellezfernandez/git/Juridique/COMPVIZ/data/dataset.zip
 57%|█████▋    | 177M/310M [00:05<00:04, 31.8MB/s] 

KeyboardInterrupt: 

On définit une classe BirdDataset afin de charger l'ensemble des données à partir du répértoire de stockage (path), le chemin vers le fichier de mapping de classes (class_mapping_path) et quelques transformations à appliquer sur les images (transforms)

In [28]:
# Define BirdDataset
class BirdDataset(Dataset):
    def __init__(self, path, class_mapping_path, transforms=None):
        super().__init__()
        self.path = path
        self.image_path = list(Path(path).glob("*/*.jpg"))
        self.class_mapping = pd.read_csv(class_mapping_path)
        self.classes = self.class_mapping.sort_values(by='idx').category_cub.to_list()
        self.transforms = transforms

    def __len__(self):
        return len(self.image_path)

    # Update the __getitem__ method in the BirdDataset class
def __getitem__(self, idx):
    image_path = self.image_path[idx]
    target = torch.tensor(self.classes.index(image_path.parent.name))
    with open(image_path, 'rb') as f:
        with Image.open(f) as img:
            image = img.convert('RGB')
    
    if self.transforms is not None:
        image_np = np.array(image)
        augmented = self.transforms(image=image_np)
        augmented_image = augmented['image']
        
        # Ensure that the augmented image has the correct shape
        if len(augmented_image.shape) == 3:
            image = Image.fromarray(augmented_image.transpose(1, 2, 0))
        else:
            raise ValueError("Augmented image has incorrect shape")
    
    return image, target, image_path.name


In [29]:
# Function to detect and crop birds
def detect_and_crop_bird(model, image):
    results = model(image)  # Run inference
    predictions = results[0].boxes.xyxy.cpu().numpy()  # Get bounding boxes
    scores = results[0].boxes.conf.cpu().numpy()  # Get confidence scores
    
    if len(predictions) > 0:
        max_proba = np.max(scores)
        bbox = predictions[np.argmax(scores), :4].astype(int)
        if max_proba > 0:
            return crop_img(image, bbox)
    return image

def crop_img(image, bbox):
    left, top, right, bottom = bbox
    width = right - left
    height = bottom - top
    max_dim = max(width, height)

    center_x = left + width // 2
    center_y = top + height // 2

    new_left = max(center_x - max_dim // 2, 0)
    new_top = max(center_y - max_dim // 2, 0)
    new_right = min(center_x + max_dim // 2, image.shape[1])
    new_bottom = min(center_y + max_dim // 2, image.shape[0])

    cropped = image[new_top:new_bottom, new_left:new_right]

    # Ensure cropped image is square
    cropped_height, cropped_width, _ = cropped.shape
    if cropped_height != cropped_width:
        min_dim = min(cropped_height, cropped_width)
        cropped = cropped[:min_dim, :min_dim]

    return cropped
    

# Function to crop and save images
def crop_and_save_images(dataset, output_path, model):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    class_dirs = {class_name: os.path.join(output_path, class_name) for class_name in dataset.classes}
    for class_dir in class_dirs.values():
        os.makedirs(class_dir, exist_ok=True)
    
    for i in tqdm(range(len(dataset))):
        image, class_idx, image_name = dataset[i]
        im = np.array(image)
        cropped = detect_and_crop_bird(model, im)
        cropped = Image.fromarray(cropped.astype('uint8'))  # Convert to uint8 before creating PIL image
        save_path = os.path.join(class_dirs[dataset.classes[class_idx]], image_name)
        cropped.save(save_path)

# Load YOLOv8 model
model_yolo = YOLO('yolov8x.pt')

# Initialize original dataset
val_dataset = BirdDataset("./data/dataset/val_images/", "./data/dataset/class_indexes.csv")

# Crop and save images
output_dataset_path = "./data/dataset/cropped_val_images"
crop_and_save_images(val_dataset, output_dataset_path, model_yolo)


  0%|          | 0/270 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 3)

: 

In [5]:
model_yolo = YOLO('yolov8x.pt')

# Initialize original dataset
train_dataset = BirdDataset("./data/dataset/train_images/", "./data/dataset/class_indexes.csv")

# Crop and save images
output_train_dataset_path = "./data/dataset/cropped_train_images"
crop_and_save_images(train_dataset, output_train_dataset_path, model_yolo)


0: 512x640 1 bird, 7.2ms
Speed: 1.2ms preprocess, 7.2ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)



0: 544x640 1 bird, 7.9ms
Speed: 0.9ms preprocess, 7.9ms inference, 0.4ms postprocess per image at shape (1, 3, 544, 640)

0: 448x640 1 bird, 6.2ms
Speed: 0.8ms preprocess, 6.2ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 bird, 6.0ms
Speed: 0.6ms preprocess, 6.0ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)



0: 448x640 1 bird, 6.0ms
Speed: 0.7ms preprocess, 6.0ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 bird, 6.0ms
Speed: 0.7ms preprocess, 6.0ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 bird, 6.0ms
Speed: 0.7ms preprocess, 6.0ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)



0: 512x640 1 bird, 7.3ms
Speed: 0.7ms preprocess, 7.3ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)

0: 448x640 1 bird, 6.2ms
Speed: 0.7ms preprocess, 6.2ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 640x448 1 bird, 6.2ms
Speed: 0.7ms preprocess, 6.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 448)



0: 480x640 1 bird, 6.4ms
Speed: 0.7ms preprocess, 6.4ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bird, 6.2ms
Speed: 0.7ms preprocess, 6.2ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 640x640 1 bird, 8.3ms
Speed: 0.9ms preprocess, 8.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


...


0: 512x640 1 bird, 6.7ms
Speed: 0.7ms preprocess, 6.7ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)

0: 416x640 1 bird, 5.8ms
Speed: 0.6ms preprocess, 5.8ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 640)

0: 512x640 1 bird, 6.7ms
Speed: 0.7ms preprocess, 6.7ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 512x640 1 bird, 6.5ms
Speed: 0.7ms preprocess, 6.5ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)

0: 480x640 1 bird, 6.0ms
Speed: 0.6ms preprocess, 6.0ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 512x640 1 bird, 6.7ms
Speed: 0.7ms preprocess, 6.7ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 640x640 1 bird, 7.7ms
Speed: 0.9ms preprocess, 7.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 bird, 6.0ms
Speed: 0.7ms preprocess, 6.0ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)



0: 448x640 1 bird, 5.8ms
Speed: 0.6ms preprocess, 5.8ms inference, 0.3ms postprocess per image at shape (1, 3, 448, 640)

0: 544x640 1 bird, 1 potted plant, 7.3ms
Speed: 0.7ms preprocess, 7.3ms inference, 0.4ms postprocess per image at shape (1, 3, 544, 640)

0: 640x576 1 bench, 1 bird, 7.0ms
Speed: 0.7ms preprocess, 7.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 576)

0: 384x640 1 bird, 5.4ms
Speed: 0.5ms preprocess, 5.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 448x640 1 bird, 5.7ms
Speed: 0.6ms preprocess, 5.7ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 bird, 5.5ms
Speed: 0.6ms preprocess, 5.5ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 2 birds, 5.6ms
Speed: 1.1ms preprocess, 5.6ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 bird, 5.5ms
Speed: 0.8ms preprocess, 5.5ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640


0: 448x640 1 bird, 5.5ms
Speed: 0.6ms preprocess, 5.5ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 bird, 5.5ms
Speed: 0.6ms preprocess, 5.5ms inference, 0.3ms postprocess per image at shape (1, 3, 448, 640)

0: 480x640 1 bird, 6.0ms
Speed: 0.7ms preprocess, 6.0ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bird, 5.8ms
Speed: 0.6ms preprocess, 5.8ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 1 bird, 5.7ms
Speed: 0.6ms preprocess, 5.7ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 544x640 1 bird, 7.3ms
Speed: 1.4ms preprocess, 7.3ms inference, 0.3ms postprocess per image at shape (1, 3, 544, 640)

0: 640x576 1 bird, 7.0ms
Speed: 0.9ms preprocess, 7.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 576)

0: 416x640 1 bird, 5.7ms
Speed: 0.6ms preprocess, 5.7ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 640)



0: 448x640 1 bird, 5.7ms
Speed: 0.6ms preprocess, 5.7ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 bird, 5.5ms
Speed: 0.6ms preprocess, 5.5ms inference, 0.4ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 bird, 5.5ms
Speed: 0.6ms preprocess, 5.5ms inference, 0.3ms postprocess per image at shape (1, 3, 448, 640)

0: 512x640 2 birds, 6.7ms
Speed: 0.7ms preprocess, 6.7ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 640)

0: 480x640 2 birds, 6.0ms
Speed: 0.7ms preprocess, 6.0ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 1 bird, 5.7ms
Speed: 0.6ms preprocess, 5.7ms inference, 0.3ms postprocess per image at shape (1, 3, 448, 640)

0: 512x640 1 bird, 6.7ms
Speed: 0.7ms preprocess, 6.7ms inference, 0.4ms postprocess per image at shape (1, 3, 512, 640)


100%|██████████| 1525/1525 [00:26<00:00, 57.54it/s]
